Write Apache Beam code to count attendance of employee. Exclude a list of employees given in side input.
The length of employee name should be betwwen 3 and 10.

Output should be in the format -

emp_id emp_name - attendance count


In [24]:
!pip install --quiet apache-beam[gcp]

In [ ]:
from google.colab import files
files.upload()

In [25]:
import apache_beam as beam
p1=beam.Pipeline()

side_list = list()
with open('/content/exclude_ids.txt') as af:
  for line in af:
    exclude_ids=line.rsplit()
    side_list.append(exclude_ids)

print(side_list)


[['149633CM'], ['212539MU'], ['231555ZZ'], ['704275DC']]


In [27]:
side_list = list()
with open('/content/exclude_ids.txt') as af:
  exclude_ids=af.read().rsplit()
  side_list.append(exclude_ids)

print(side_list)

[['149633CM', '212539MU', '231555ZZ', '704275DC']]


In [26]:
from apache_beam.io.textio import WriteToText
import apache_beam as beam
p1=beam.Pipeline()

side_list = list()
with open('/content/exclude_ids.txt') as af:
  exclude_ids=af.read().rsplit()
  side_list.append(exclude_ids)

class FilterUsingLength(beam.DoFn):
  def process(self,record,side_list,lower_bound,upper_bound):
    element_list=record.split(',')
    emp_id=element_list[0]
    emp_name=element_list[1]

    if len(emp_name)>lower_bound and len(emp_name)<upper_bound:
      if emp_id not in side_list and element_list[3]=='Accounts':
        return [element_list]

# print(side_list)
exclude_employee = (p1
                    | beam.io.ReadFromText('/content/dept_data.txt')
                    | beam.ParDo(FilterUsingLength(),side_list,3,10)
                    |beam.Map(lambda record: (record[0]+" "+record[1],1))
                    |beam.CombinePerKey(sum)
                    |beam.io.WriteToText('/content/output-t'))

p1.run()

!{('head -n 10 /content/output-t*')}



('149633CM Marco', 31)
('212539MU Rebekah', 31)
('231555ZZ Itoe', 31)
('503996WI Edouard', 31)
('704275DC Kyle', 31)
('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('718737IX Ayumi', 30)
